*Alejandro Corrochano's contribution to the final project.* 

In [17]:
# Common use 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import time

#Metrics
import sklearn.metrics as metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

#RDKIT
from rdkit import Chem

#Feature selection
from mrmr import mrmr_classif
from sklearn.feature_selection import SelectKBest
from skrebate import ReliefF
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif

#Models - Regression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

#Models - Classification
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

## Visualization

In [2]:
def data_overview(df, split):
    print('Total number of molecules:',len(df))
    print('Train set: %d - %0.1f%%' %(len(split['train']), len(split['train'])/len(df)*100))
    print('Validation set: %d - %0.1f%%' %(len(split['valid']), len(split['valid'])/len(df)*100))
    print('Test set: %d - %0.1f%%' %(len(split['test']), len(split['test'])/len(df)*100))
    display(split['train'].head())

In [3]:
def plot_comparison(names, met, c):
    sns.set(font_scale = 1)
    # set width of bar
    barWidth = 0.15
    plt.figure(figsize=(15, 6), dpi = 95)

    # set height of bar
    var1 = [i[0] for i in met]
    var2 = [i[1] for i in met]
    var3 = [i[2] for i in met]
    if c == True:
        var4 = [i[3] for i in met]
        var5 = [i[4] for i in met]

    # Set position of bar on X axis
    br1 = np.arange(len(var1))
    br2 = [x + barWidth for x in br1]
    br3 = [x + barWidth for x in br2]
    br4 = [x + barWidth for x in br3]
    br5 = [x + barWidth for x in br4]

    # Make the plot
    if c == False:
        plt.bar(br1, var1, color ='r', width = barWidth,
                edgecolor ='grey', label ='R2')
        plt.bar(br2, var2, color ='g', width = barWidth,
                edgecolor ='grey', label ='MAE')
        plt.bar(br3, var3, color ='b', width = barWidth,
                edgecolor ='grey', label ='MSE')
    
    # Add 2 more bars 
    else :
        plt.bar(br1, var1, color ='r', width = barWidth,
                edgecolor ='grey', label ='MCC')
        plt.bar(br2, var2, color ='g', width = barWidth,
                edgecolor ='grey', label ='AUC')
        plt.bar(br3, var3, color ='b', width = barWidth,
                edgecolor ='grey', label ='ACC')
        plt.bar(br4, var4, color ='yellow', width = barWidth,
            edgecolor ='grey', label ='PRE')
        plt.bar(br5, var5, color ='purple', width = barWidth,
            edgecolor ='grey', label ='REC')
    
    # Adding Xticks
    plt.xlabel('Models', fontweight ='bold', fontsize = 15)
    plt.ylabel('Metrics', fontweight ='bold', fontsize = 15)
    plt.xticks([r + barWidth for r in range(len(var1))],
            [name for name in names])
    plt.legend()
    plt.title('Metrics obtained for each model')
    
    plt.show()

In [4]:
def plot_reg_fs_metrics(num_sel_feat, results, fs_name):
    
    mse, r2 = [], []
    for iteration in results:
        mse.append(min([mse[2] for mse in iteration]))
        r2.append(max([r2[0] for r2 in iteration]))
    
    # Plot R2 and MSE 
    plt.figure(figsize=(10, 4), dpi = 95)
    sns.set_theme(style="darkgrid")

    sns.lineplot(x = np.arange(10, max(num_sel_feat)+5, 5), y = mse, marker='o', label = 'MSE')
    sns.lineplot(x = np.arange(10, max(num_sel_feat)+5, 5), y = r2, marker='o', label = 'R2')
    
    # Vertical line to indicate best performance
    sns.lineplot([num_sel_feat[np.argmax(r2)], num_sel_feat[np.argmax(r2)]], [0, max(mse)], 
                 color = 'red',linewidth = 4, label = 'Best')
    plt.legend()
    plt.title('MSE and R2 scores vs number of best features selected')
    plt.xlabel('Number of selected features by %s'%fs_name)
    plt.ylabel('Metrics')
    plt.show()
    return mse, r2

## Functionality

In [5]:
def remove_ha_duplicates(df):
    og_shape = df.shape
    heavy_atoms = pd.Series([Chem.MolFromSmiles(smi).GetNumHeavyAtoms() for smi in df['Drug']])
    # DF now only contains compounds with 5 or more heavy atoms
    df = df[heavy_atoms >= 5]
    # Duplicates removal from DF
    df.drop(df[df['Drug'].duplicated()].index, inplace = True)
    
    print('Duplicated compounds and with less than 5 heavy atoms have been removed.')
    print('New number of compounds: %d (%d)' %(df.shape[0], (df.shape[0] - og_shape[0])))
    return df

In [8]:
def train_val_test_split(df, split):
    
    # Training
    X_train = df[df.columns[3:-1]][:len(split['train'])]
    y_train = df['Y'][:len(split['train'])]
    #Same with fingerprints (Since Y is the same, we just need to care of X)
#     X_train_fps = fps[fps.columns[:]][:len(split['train'])]
    
    
    # Validation
    X_val = df[df.columns[3:-1]][len(split['train']):(len(split['valid'])+len(split['train']))]
    y_val = df['Y'][len(split['train']):(len(split['valid'])+len(split['train']))]
    #Same with fingerprints (Since Y is the same, we just need to care of X)
#     X_val_fps = fps[fps.columns[:]][len(split['train']):(len(split['valid'])+len(split['train']))]

    
    # Test
    X_test = df[df.columns[3:-1]][(len(split['valid'])+len(split['train'])):len(df)]
    y_test = df['Y'][(len(split['valid'])+len(split['train'])):len(df)]
    #Same with fingerprints (Since Y is the same, we just need to care of X)
#     X_test_fps = fps[fps.columns[:]][(len(split['valid'])+len(split['train'])):len(df)]
    
    print('Data has been split')
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [7]:
# Remove molecules that contain many zero-value/nan features
def remove_nans(X_train, X_val, X_test, y_train, y_val, y_test):
    for ind, dset in enumerate([X_train, X_val, X_test]):
        nan_mols = dset.isnull().any(axis=1)
        if len(dset[nan_mols].index) !=0 :
            # Check rows where NaN values are located 
            nan_rows = dset[nan_mols].index
            # Remove NaNs in dataset
            dset.drop(index = nan_rows, inplace = True)
            if ind == 0:
                y_train.drop(index = nan_rows, inplace = True)
#                 X_train_fps.drop(index = nan_rows, inplace = True)
                print('Removed the following rows in the train set:',nan_rows)

            elif ind == 1:
#                 X_val_fps.drop(index = nan_rows, inplace = True)
                y_val.drop(index = nan_rows, inplace = True)
                print('Removed the following rows in the val set:',nan_rows)

            else:
                y_test.drop(index = nan_rows, inplace = True)
                print('Removed the following rows in the test set:',nan_rows)
        
    return X_train, y_train, X_val, y_val, X_test, y_test

In [9]:
def normalize_data(X_train, X_val, X_test):
    scaler = StandardScaler()
    
    # convert in into DF after standardizing 
    X_train_norm = scaler.fit_transform(X_train)
    X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
    
    X_val_norm = scaler.transform(X_val)
    X_val_norm = pd.DataFrame(X_val_norm, columns = X_val.columns)
    
    X_test_norm = scaler.transform(X_test)
    X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)
    
    print('Data is now normalized.')
    
    return X_train_norm, X_val_norm, X_test_norm

### Feature selection algorithms

In [10]:
def fs_mrmr(X_train, y_train, X_val, y_val, X_train_norm, X_val_norm, c = False):
    num_sel_feat = np.arange(10, X_train.shape[1], 5)
    m,r,f = [], [], []
    for k in num_sel_feat:
        selected_features = mrmr_classif(X_train, y_train, K = k)
        f.append(selected_features)
        # Get metrics based on selected features
        models, results = models_comparison(X_train[selected_features], y_train, X_val[selected_features], y_val,
                                        c, False, False, X_train_norm[selected_features], X_val_norm[selected_features])
        m.append(models)
        r.append(results)
    
    return m,r,num_sel_feat,f

In [11]:
def fs_score_fn(X_train, y_train, X_val, y_val, X_train_norm, X_val_norm, c, score_fn):
    
    num_sel_feat = np.arange(10, X_train.shape[1], 5)
    
    m,r,f = [], [], []
    
    for k in list(num_sel_feat):
        print("\n=======================Selected features %0.0f/%0.0f ======================="%(k,X_train.shape[1]))
        
        # Applying feature selection algorithm supplied by the score_func on train, validation and normalized datasets
        fs = SelectKBest(score_func=score_fn, k=k)         
        fs.fit(X_train, y_train)
                
        # Retreive selected feature names
        selected_features=X_train.columns[fs.get_support()]
        
        f.append(selected_features) 
        
        # Get metrics based on selected features
        models, results = models_comparison(X_train[selected_features], y_train, X_val[selected_features], y_val,
                            c, False, False, X_train_norm[selected_features], X_val_norm[selected_features])
        
        m.append(models)
        r.append(results)
         
    return m,r,num_sel_feat,f 

In [12]:
def fs_relieff(X_train, y_train, X_val, y_val, X_train_norm, X_val_norm, c, n_neighbors):
    
    num_sel_feat = np.arange(10, X_train.shape[1], 5)
    
    m,r,f = [], [], []
    
    for k in list(num_sel_feat):
        print("\n=======================Selected features %0.0f/%0.0f ======================="%(k,X_train.shape[1]))
        # Applying relieff feature selection on train, validation and normalized datasets
        fs = ReliefF(n_features_to_select=k, n_neighbors=n_neighbors)
        
        fs.fit_transform(X_train.to_numpy(), y_train.to_numpy())
        pos = pd.DataFrame(fs.feature_importances_.reshape(-1,1)).sort_values(by=0, ascending=False).head(k).index.tolist()
        selected_features = list(X_train.columns[pos])

        f.append(selected_features) 
        
        # Get metrics based on selected features
        models, results = models_comparison(X_train[selected_features], y_train, X_val[selected_features], y_val,
                            c, False, False, X_train_norm[selected_features], X_val_norm[selected_features])
                
        m.append(models)
        r.append(results)
        
    return m,r,num_sel_feat,f 

### Models comparison

In [13]:
# Compute regression metrics 
def get_metrics(model_name, y_true, y_pred, c, train_times, mode = 0):
    # Compute metrics 
    if c == False:
        var1 = metrics.r2_score(y_true, y_pred)
        var2 = metrics.mean_absolute_error(y_true, y_pred)
        var3 = metrics.mean_squared_error(y_true, y_pred)
        
        print(model_name,'| R2: %0.3f, MAE: %0.3f, MSE: %0.3f | Training time: %0.1f' %(var1, var2, var3, train_times[-1]))
        
        # Mode to 1 displays r-squared plots
        if mode == 1: 
            plt.figure(figsize=(10, 5), dpi = 95)

            plt.scatter(y_true, y_pred, color='salmon', s=5)
            plt.plot(np.unique(y_true), np.poly1d(np.polyfit(y_true, y_pred, 1))(np.unique(y_true)), color='black')

            plt.text(0, 3.5,'R-squared = %0.2f' % var1)
            plt.xlabel('Actual values')
            plt.ylabel('Predicted Values')
            plt.title('Prediction results using {}'.format(model_name))
            plt.show()
        
        return var1, var2, var3
        
    else:
        var1 = metrics.matthews_corrcoef(y_true, y_pred)
        var2 = metrics.roc_auc_score(y_true, y_pred)
        var3 = metrics.precision_score(y_true, y_pred)
        var4 = metrics.recall_score(y_true, y_pred)
        var5 = metrics.accuracy_score(y_true, y_pred)
        
        print(model_name,'| MCC: %0.3f, AUC: %0.3f, Accuracy: %0.3f, Precision: %0.3f, Recall: %0.3f | Training time: %0.1f' 
              %(var1, var2, var3, var4, var5, train_times[-1]))
        
    return var1, var2, var3, var4, var5

In [14]:
def models_comparison(X_train, y_train, X_val, y_val, c, plot = True, fps = False 
                          , X_train_norm = 0, X_val_norm = 0):
    np.random.seed(10)
    n_models = ['LR', 'RFR', 'DTR', 'SVR', 'MLPR', 'XGBR', 'MLPC', 'SVC', 'RFC', 'DTC', 'XGBC']
    models = {'Regression': 
              {'Linear': {'LR': LinearRegression(), 'SVR': SVR(), 
                          'MLPR': MLPRegressor(hidden_layer_sizes=(128,64,32), max_iter=500)},
               'Non-linear':{'RFR': RandomForestRegressor(), 'DTR': DecisionTreeRegressor(), 'SVR':SVR(), 'XGBR': XGBRegressor()}},
                  
              'Classification': 
              {'Linear': {'SVC': SVC(), 
                          'MLPC': MLPClassifier(hidden_layer_sizes=(128,64,32), max_iter=500)},
               'Non-linear':{'RFC': RandomForestClassifier(), 'DTC': DecisionTreeClassifier(), 'XGBC': XGBClassifier()}}
             }
             

    names, results, train_times = [], [], []

    for name in n_models:
        #REGRESSION
        if name in models['Regression']['Linear'].keys() and c == False:
            # Train the model and measure the time required
            start_time = time.time()
            clf = models['Regression']['Linear'][name].fit(X_train_norm, y_train)
            train_time = time.time() - start_time
            # Predict validation set values
            y_pred = clf.predict(X_val_norm)
            # Store the name of the model, metrics, and time for later visualization
            names.append(name)
            train_times.append(train_time)
            r2, mae, mse = get_metrics(name, y_val, y_pred, c, train_times)
            results.append([r2, mae, mse])
            
        elif name in models['Regression']['Non-linear'].keys() and c == False:
            
            start_time = time.time()
            clf = models['Regression']['Non-linear'][name].fit(X_train, y_train)
            train_time = time.time() - start_time
            
            y_pred = clf.predict(X_val)
            
            names.append(name)
            train_times.append(train_time)
            r2, mae, mse = get_metrics(name, y_val, y_pred, c, train_times)
            results.append([r2, mae, mse])
            
            
        #CLASSIFICATION
        elif name in models['Classification']['Linear'].keys() and c == True:
            start_time = time.time()
            clf = models['Classification']['Linear'][name].fit(X_train_norm, y_train)
            train_time = time.time() - start_time
            
            y_pred = clf.predict(X_val_norm)
            
            names.append(name)
            train_times.append(train_time)
            mcc, auc, acc, pre, rec = get_metrics(name, y_val, y_pred, c, train_times)
            results.append([mcc, auc, acc, pre, rec])
            
        elif name in models['Classification']['Non-linear'].keys() and c == True :
            start_time = time.time()
            clf = models['Classification']['Non-linear'][name].fit(X_train, y_train)    
            train_time = time.time()
            
            y_pred = clf.predict(X_val)

            names.append(name)
            train_times.append(train_time)
            mcc, auc, acc, pre, rec = get_metrics(name, y_val, y_pred, train_times, c, train_times)
            results.append([mcc, auc, acc, pre, rec])
            
            
        else:
            pass
        
    # We plot the metrics obtained for each model
    if plot == True:
        plot_comparison(names, results, c)
    return names, results

### Hyperparameter tuning

In [15]:
def apply_randsearch(X_train, y_train, X_val, y_val, c, X_train_norm=0, X_val_norm=0):
    np.random.seed(10)
    
    #---------------MODELS---------------------------------------
    # REGRESSION
    #RandomForestRegressor parameters
    '''grid_param_rf = {
         'n_estimators': [200, 250, 300],
         'max_depth': [10, 20, 30, 40, None],
         'max_features': ['auto', 'sqrt', 'log2'],
         'min_samples_split': [2, 5, 10, 15],
         'min_samples_leaf': [1, 2, 5, 10],
         'bootstrap': [True, False]
     }'''

    #SVR parameters
    grid_param_svm = {'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
                      'C' : [1,5,10],
                      'degree' : [3,8],
                      'coef0' : [0.01,10,0.5],
                      'gamma' : ('auto','scale')
                     }
    
    #XGBRegressor parameters
    '''grid_param_xgb = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'min_child_weight': [3, 5, 8, 10],
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 200, 500, 700, 1000]}'''

    #--------------------------------------------------------------------------
    
    rnd_models = []
    if c is True:
         #[(grid_param_svm, SVC(), 1), (grid_param_rf, RandomForestClassifier(), 1), (grid_param_xgb, XGBClassifier(), 1)]
        params = [(grid_param_svm, SVC(), 0)]
    else:
        #[(grid_param_svr, SVR(), 1), (grid_param_rf, RandomForestRegressor(), 1), (grid_param_xgb,XGBRegressor(), 1)]
        params = [(grid_param_svm, SVR(), 1)]
    
    for grid in params:
        random_config = RandomizedSearchCV(grid[1], param_distributions = grid[0], n_iter = 20, cv = 5, n_jobs = -1)
        rnd_models.append((random_config, grid[2]))
      
    # Fit all the models and store their predictions
    y_preds = []
    for ind, m in enumerate(rnd_models):
        # Make a distinction between linear and non linear models, 1 and 0 respectively 
        if m[1] == 0:
            rnd_models[ind] = m[0].fit(X_train, y_train)
            temp_clf = rnd_models[ind].best_estimator_
            print(rnd_models[ind].best_params_)
            y_preds.append(temp_clf.predict(X_val))
            
        elif m[1] == 1:
            rnd_models[ind] = m[0].fit(X_train_norm, y_train)
            temp_clf = rnd_models[ind].best_estimator_
            print(rnd_models[ind].best_params_)
            y_preds.append(temp_clf.predict(X_val_norm))
    
    #METRICS
    if c is True:
        #Compute classification metrics 
        mcc = metrics.matthews_corrcoef(y_true, y_pred)
        auc = metrics.roc_auc_score(y_true, y_pred)
        prec = metrics.precision_score(y_true, y_pred)
        rec = metrics.recall_score(y_true, y_pred)
        acc = metrics.accuracy_score(y_true, y_pred)
        #Store metrics to return them later
        mets = (mcc, auc, prec, rec, acc)
        print('MCC: %f, AUC: %f, PRECISION: %f, RECALL: %f, ACCURACY: %f' %(mcc, auc, prec, rec, acc))
    else:
        #Compute regression metrics 
        for preds in y_preds:
            r2 = metrics.r2_score(y_val, preds)
            mae = metrics.mean_absolute_error(y_val, preds)
            mse = metrics.mean_squared_error(y_val, preds)
            #Store metrics to return them later
            mets = (r2, mse, mae)
            print('R2: %f, MSE: %f, MAE: %f' %(r2, mse, mae))

    return rnd_models, mets

### Fingerprints

In [16]:
# FP_data = (num_mols x numVectorBits)
def generate_fingerprints(mol, fp_type="ECFP", r = 2, bits =1024):
    fp_arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(AllChem.GetMorganFingerprintAsBitVect(mol,r,bits),fp_arr)    
    return fp_arr

# Generate fingerprints dataset and transform into a dataframe
# fps = pd.DataFrame([generate_fingerprints(Chem.MolFromSmiles(mol)) for mol in df['Drug']],columns = np.arange(1,1025))
# print('Shape of the actual fps structure:', fps.shape)